In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ee
import geemap
import seaborn as sns
from datetime import datetime

from validate_dowscaled_et import prepare_data_for_plotting

In [ ]:
ee.Initialize(project="thurgau-irrigation")

In [ ]:
def set_negative_to_zero(image):
    return image.where(image.lt(0), 0)

In [ ]:
et_collection_2018 = ee.ImageCollection([])

for i in range(1, 13):
    et_raster = ee.Image(f"projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_downscaled/WaPOR_ET_downscaled_2018-{i}")
    et_raster = set_negative_to_zero(et_raster)
    et_collection_2018 = et_collection_2018.merge(et_raster)

ee.Date(et_collection_2018.first().date()).format("YYYY-MM-dd").getInfo()

In [ ]:
et_collection_2018.first().bandNames().getInfo()

In [ ]:
validation_et_collection = ee.FeatureCollection("projects/thurgau-irrigation/assets/Thurgau/ETFluxNet/ET_timeseries_rietholzbach_2009_2015")

In [ ]:
point = validation_et_collection.first().geometry()

Map = geemap.Map()
Map.centerObject(point, 12)
Map.addLayer(point, {"color": "red"}, "point")
Map

In [ ]:
df = prepare_data_for_plotting(validation_et_collection)

In [ ]:
def extract_pixel_values(
    image_collection: ee.ImageCollection, point: ee.Geometry.Point
) -> ee.FeatureCollection:
    """
    Extract the pixel value of the 'downscaled' band for each image in the collection
    at the specified point.

    Args:
        image_collection (ee.ImageCollection): The input image collection.
        point (ee.Geometry.Point): The point at which to extract values.

    Returns:
        ee.FeatureCollection: A feature collection where each feature represents an image
                              and contains the pixel value of the 'downscaled' band at the point.
    """

    def extract_value(image: ee.Image) -> ee.Feature:
        pixel_value = (
            image.select("downscaled")
            .reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=point,
                scale=image.projection().nominalScale(),
                bestEffort=True,
            )
            .get("downscaled")
        )

        return ee.Feature(
            None,
            {
                "pixel_value": pixel_value,
                "image_id": image.id(),
                "system:time_start": image.get("system:time_start"),
            },
        )

    # Map the extraction function over the image collection
    return ee.FeatureCollection(image_collection.map(extract_value))


raster_collection = extract_pixel_values(
    et_collection_2018, validation_et_collection.first().geometry()
)

In [ ]:
raster_collection.getInfo()